# Nuclei Mask Interactors
Updated on 2020-10-01

Interactors for comparing nuclei object masks from various methods. Nuclei masks are obtained by analyzing the DAPI channel of the CyCIF datasets (multiplex images)

Parameters are set in the configs_interactive.py file -- look there for further information.

**NOTE** - Begin the notebook by clicking on the "*Run all initialization cells*" button in the toolbar (the calculator button).


**Temporary behaviour** - currently the behaviour of the notebook only works with one image and its masks, the development of its other function is to come soon which is to work directly from a csv file that contains multiple images and associated masks. The user will be allowed to cycle between the images.

## Table of Contents
1. [Visualize Image with Target and Single Prediction](#section1)
2. [Detail Comparison of Agreement of Target and Prediction](#section2)
3. [Visualize Single Nuclei](#section3)
4. [Visualize Multiple Predictions](#section4)
5. [Cell Type Calling Feature Comparison](#section5)
6. [Cell Type Calling Feature Thresholds](#section6)
7. [ROC Curves](#section7)
8. [Feature Importance](#section8)

In [1]:
# configure the notebook
%matplotlib inline  
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('..')
sys.path.append('/data/')

import configs as ci
from pandas import read_csv
from imageio import imread
import numpy as np
import ipywidgets as widgets

from utils import ObjectsClass, ObjectClass, ObjectTypeClass
from utils.utils import normalize_image

# tentative
import matplotlib.pyplot as plt

In [ ]:
# prepare all variables
im_df = read_csv(ci.CSV_FILEPATH)

# temporarily reading a single image and its masks
i = 2
im = imread(im_df.loc[i, ci.IM_COL])
target_mask = np.load(im_df.loc[i, ci.TARGET_COL])
pred_mask1 = np.load(im_df.loc[i, ci.PRED_COLS[0]])
pred_mask2 = np.load(im_df.loc[i, ci.PRED_COLS[1]])

objComparator = ObjectClass(im, target_mask, pred_mask2, verbose=0)
objsComparator = ObjectsClass(im, target_mask, [target_mask, pred_mask1, pred_mask2], verbose=0)

<a id='section1'></a>
## 1) Visualize Image with Target and Single Prediction

In [ ]:
# 
objComparator.report_metrics()
objComparator.visualize_interact()

<a id='section2'></a>
## 2) Detail Comparison of Agreement of Target and Prediction

In [ ]:
# 
objComparator.visualize_comparison_interact()

<a id='section3'></a>
## 3) Visualize Single Nuclei

In [ ]:
#
objComparator.visualize_object_interact()

<a id='section4'></a>
## 4) Visualize Multiple Predictions

In [ ]:
#
objsComparator.visualize_heatmap_interact()

<a id='section5'></a>
## 5) Cell Type Calling Feature Comparison

In [ ]:
# prepare parameters for this section
# cellType_im = normalize_image(imread('/data/testDatasets/lung2CellTypes/regionImg_x-8000_y-1000.png'))
# cellType_mask = np.load('/data/testDatasets/lung2CellTypes/regionMask_x-8000_y-1000.npy')
# cellType_filepath = '/data/ome.tiffs/Lung2BR/Lung2.csv'

pred_dict = {'Lung3 - Morpho': '../Datafiles/xgboost_Lung3_morpho.pred.csv',
             'Lung3 - Morpho + DAPI': '../Datafiles/xgboost_Lung3_morpho_dapi.pred.csv',
             'Lung3 - Morpho + DAPI + Neighbor': '../Datafiles/xgboost_Lung3_morpho_dapi_neighbor.pred.csv'}
ft_dict = {
    'Lung3 - Morpho': '../Datafiles/xgboost_Lung3_morpho.model.feat_impt.csv', 
    'Lung3 - Morpho + DAPI': '../Datafiles/xgboost_Lung3_morpho_dapi.model.feat_impt.csv',
    'Lung3 - Morpho + DAPI + Neighbor': '../Datafiles/xgboost_Lung3_morpho_dapi_neighbor.model.feat_impt.csv'
}

region = {'left': 8000, 'top': 1000, 'width': 500, 'height': 500}
objclass = ObjectTypeClass(
    '/data/ome.tiffs/Lung3PR/LUNG-3-PR_40X.ome.tif',
    '/data/ome.tiffs/Lung3PR/LUNG-3-PR_40X_Seg_labeled.tif',
    region,
    '/data/ome.tiffs/Lung3PR/Lung3.csv',
    chname_filepath='../Datafiles/LungChNames.txt',
    pred_filepaths=pred_dict,
    markers_filepath='../Datafiles/LUNG-3-PR_40X_features.csv',
    ft_importance_filepaths=ft_dict
)
objclass.visualize_features_interact()

<a id='section6'></a>
## 6) Cell Type Calling Feature Thresholds

In [ ]:
#
objclass.visualize_threshold_interact()

## 7) Receiver operation curves
Read from csv files provided.

In [ ]:
objclass.interact_class_roc()

<a id='section8'></a>
## 8) Feature imporance
Read from csv files.

In [ ]:
# plot the feature importances
objclass.interact_ft_importance()

## 9) Marker expression and feature comparison
Each nuclei is defined by a set of pixels in the image. Given a marker channel we can calculate the average marker value for all pixels in that nuclei and repeat for all channels and nuclei in the image. This is a time consuming operation and should be done ahead of time and results saved to a csv file.

The csv file should contain CellID column that matches the cell id of other csv files.

This interactor should compare a set of marker features vs a single morphological feature. These should be scatter plots with the marker features on the y-axis and the morphological feature on x-axis.

In [ ]:
# choose a set of markers to plot against a feature
objclass.interact_ft_vs_markers()

In [ ]:
objclass.interact_hm_ft_vs_markers()

## Old version of interactors (no longer planned to use)

In [ ]:
"""ROC (old version)

The CSV contains probability for the classes for each cell - not sure where these come from but I think these are how the ground truth was generated. Doing 
ROCs of these is not that informative as it will be a near perfect curve. You can use the other features in the CSV (such as area) to predict new labels 
probabilities and then do ROC based on those or add new CSV files that contain different probability predictions.
"""

# predict the labels from a list of features
feature_set1 = ['Area',  'Eccentricity', 'Solidity', 'Extent', 'EulerNumber', 'Perimeter', 'MajorAxisLength', 'MinorAxisLength', 'Orientation',
                'Neighbor_1', 'Neighbor_2', 'Neighbor_3', 'Neighbor_4', 'Neighbor_5', 'X_position', 'Y_position']
feature_set2 = ['Perimeter', 'Extent', 'Orientation']
objclass.predict_labels('All features', feature_set1)
print()
objclass.predict_labels('3 features', feature_set2)
print()
objclass.predict_labels('All fts + 300 est', feature_set1, n_estimators=300)

objclass.predict_labels('All fts + 300 est', feature_set1, n_estimators=300, full=False)

# plot the ROC curve(s)
objclass.interact_roc()